## 贝叶斯网络 (Bayesian Network)
使用 pgmpy 库构建和推理贝叶斯网络：

In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
import pandas as pd

# Create a toy dataset
data = pd.DataFrame({
    'Disease': [1, 1, 0, 1, 0, 0, 1],
    'Cough': [1, 0, 0, 1, 0, 0, 1],
    'Fever': [1, 1, 0, 1, 1, 0, 0]
})

# Define the Bayesian Network structure
model = BayesianNetwork([('Disease', 'Cough'), ('Disease', 'Fever')])

# Fit the model using maximum likelihood estimation
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Perform inference
inference = VariableElimination(model)
result = inference.map_query(variables=['Cough'], evidence={'Disease': 1})
print("Inference result:", result)


INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Inference result: {'Cough': 1}


## 隐马尔可夫模型 (Hidden Markov Model, HMM)
使用 hmmlearn 库实现隐马尔可夫模型：

In [2]:
from hmmlearn import hmm
import numpy as np

# Create a toy dataset for HMM
X = np.array([[1], [2], [3], [4], [5], [6]])
lengths = [len(X)]

# Define and train an HMM model
model = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100, random_state=42)
model.fit(X, lengths)

# Predict the hidden states
hidden_states = model.predict(X)
print("Hidden states:", hidden_states)


Hidden states: [0 1 1 2 2 2]


## 马尔可夫随机场 (Markov Random Fields, MRF)
pgmpy 可用于构建无向图模型来表示马尔可夫随机场：

In [14]:
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
import numpy as np

# 创建马尔可夫网络
model = MarkovNetwork()

# 添加节点
model.add_nodes_from(['A', 'B', 'C'])

# 添加边 (定义节点之间的连接)
model.add_edges_from([('A', 'B'), ('B', 'C')])

# 定义节点的单变量势函数
# A 有 2 个状态 (0 和 1)
phi_a = DiscreteFactor(['A'], cardinality=[2], values=[0.6, 0.4])

# B 有 2 个状态 (0 和 1)
phi_b = DiscreteFactor(['B'], cardinality=[2], values=[0.5, 0.5])

# 定义边的联合势函数
# A 和 B 的势函数
phi_ab = DiscreteFactor(['A', 'B'], cardinality=[2, 2], values=[1.0, 0.3, 0.3, 1.0])

# B 和 C 的势函数
phi_bc = DiscreteFactor(['B', 'C'], cardinality=[2, 2], values=[1.0, 0.2, 0.2, 1.0])

# 添加势函数到模型
model.add_factors(phi_a, phi_b, phi_ab, phi_bc)

# 检查模型结构和势函数是否正确
model.check_model()

# 打印模型信息
print("Model:", model)
print("Factors:", model.get_factors())

# 计算某个特定分配的联合概率
# 例如 P(A=0, B=1, C=0)
assignments = {'A': 0, 'B': 1, 'C': 0}
joint_prob = phi_a.reduce([('A', assignments['A'])], inplace=False) * \
              phi_b.reduce([('B', assignments['B'])], inplace=False) * \
              phi_ab.reduce([('A', assignments['A']), ('B', assignments['B'])], inplace=False) * \
              phi_bc.reduce([('B', assignments['B']), ('C', assignments['C'])], inplace=False)

# 计算联合概率值
joint_prob_value = joint_prob.values
print(f"Joint Probability for {assignments}: {joint_prob_value:.4f}")


Model: MarkovNetwork with 3 nodes and 2 edges
Factors: [<DiscreteFactor representing phi(A:2) at 0x20f988e0a90>, <DiscreteFactor representing phi(B:2) at 0x20f988e16d0>, <DiscreteFactor representing phi(A:2, B:2) at 0x20f988e3a90>, <DiscreteFactor representing phi(B:2, C:2) at 0x20f988e0dd0>]
Joint Probability for {'A': 0, 'B': 1, 'C': 0}: 0.0180


### **模型特点、适用场景及优缺点**

#### **1. 贝叶斯网络**
- **特点**：基于有向图表示变量之间的因果关系。
- **适用场景**：诊断问题、决策支持、风险分析。
- **优点**：
  - 能清晰表示因果关系。
  - 高效推理复杂的概率关系。
- **缺点**：
  - 网络结构需要人工构造或学习，较复杂。
  - 高维数据中，推理和训练成本较高。

---

#### **2. 隐马尔可夫模型 (HMM)**
- **特点**：模型包含隐变量，适合时间序列数据。
- **适用场景**：语音识别、基因序列分析、时间序列预测。
- **优点**：
  - 能处理隐含的动态过程。
  - 易于实现和扩展。
- **缺点**：
  - 假设状态间独立性，不适合所有数据。
  - 参数估计复杂，需大量数据。

---

#### **3. 马尔可夫随机场 (MRF)**
- **特点**：基于无向图表示变量间的条件独立关系。
- **适用场景**：图像分割、语义分析、自然语言处理。
- **优点**：
  - 灵活性强，适合复杂的无向依赖关系。
  - 能处理高维数据。
- **缺点**：
  - 推理和学习成本较高。
  - 难以确定和学习潜在的图结构。

---

### **选型建议**
- **因果关系分析**：选择贝叶斯网络。
- **时间序列分析**：选择隐马尔可夫模型。
- **图像/语义分析**：选择马尔可夫随机场。

